In [1]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from pyspark.sql.functions import from_json, explode, col


StatementMeta(, c0ce893d-328f-4dd4-9b03-08c2e3ccf694, 3, Finished, Available, Finished)

In [2]:
species_count_schema = StructType([
    StructField('total_results',IntegerType(),True),  
    StructField('page',IntegerType(),True),  
    StructField('per_page',IntegerType(),True),
    StructField('results', ArrayType(
        StructType([
            StructField('count',IntegerType(),True),  
            StructField('taxon', StructType([
                StructField('id',IntegerType(),True),  
                StructField('rank',StringType(),True),  
                StructField('rank_level',IntegerType(),True),  
                StructField('iconic_taxon_id',IntegerType(),True),  
                StructField('ancestor_ids',ArrayType(IntegerType()),True),  
                StructField('is_active',BooleanType(),True),  
                StructField('name',StringType(),True),  
                StructField('parent_id',IntegerType(),True),  
                StructField('ancestry',StringType(),True),  
                StructField('extinct',BooleanType(),True),  
                StructField('default_photo',StructType([StructField('id',IntegerType(),True),  
                StructField('license_code',StringType(),True),  
                StructField('attribution',StringType(),True),  
                StructField('url',StringType(),True),  
                StructField('original_dimensions',StructType([StructField('height',IntegerType(),True),  
                StructField('width',IntegerType(),True)]),True),  
                StructField('flags',ArrayType(StructType([StructField('id',IntegerType(),True),  
                StructField('flag',StringType(),True),  
                StructField('comment',StringType(),True),  
                StructField('user_id',IntegerType(),True),  
                StructField('resolver_id',IntegerType(),True),  
                StructField('resolved',BooleanType(),True),  
                StructField('created_at',StringType(),True),  
                StructField('updated_at',StringType(),True)])),True),  
                StructField('square_url',StringType(),True),  
                StructField('medium_url',StringType(),True)]),True),  
                StructField('taxon_changes_count',IntegerType(),True),  
                StructField('taxon_schemes_count',IntegerType(),True),  
                StructField('observations_count',IntegerType(),True),  
                StructField('flag_counts',StructType([StructField('resolved',IntegerType(),True),  
                StructField('unresolved',IntegerType(),True)]),True),  
                StructField('current_synonymous_taxon_ids',StringType(),True),  
                StructField('atlas_id',StringType(),True),  
                StructField('complete_species_count',StringType(),True),  
                StructField('wikipedia_url',StringType(),True),  
                StructField('complete_rank',StringType(),True),  
                StructField('iconic_taxon_name',StringType(),True),  
                StructField('preferred_common_name',StringType(),True)
            ]))
        ])
    ))
])


StatementMeta(, c0ce893d-328f-4dd4-9b03-08c2e3ccf694, 4, Finished, Available, Finished)

In [7]:
filePath = '[REDACTED]/iNaturalist'

df = spark.read.schema(species_count_schema).json(filePath)
df = df.select('page','total_results','per_page', explode("results").alias("results"))
df = df.select('page','results.count', 'results.taxon.id', 'results.taxon.name', 'results.taxon.extinct', 
    'results.taxon.iconic_taxon_name', 'results.taxon.preferred_common_name','results.taxon.wikipedia_url', 
    'results.taxon.observations_count', 'results.taxon.default_photo.url')

StatementMeta(, c0ce893d-328f-4dd4-9b03-08c2e3ccf694, 9, Finished, Available, Finished)

In [8]:
df.show()

StatementMeta(, c0ce893d-328f-4dd4-9b03-08c2e3ccf694, 10, Finished, Available, Finished)

+----+-----+---+----+-------+-----------------+---------------------+-------------+------------------+---+
|page|count| id|name|extinct|iconic_taxon_name|preferred_common_name|wikipedia_url|observations_count|url|
+----+-----+---+----+-------+-----------------+---------------------+-------------+------------------+---+
+----+-----+---+----+-------+-----------------+---------------------+-------------+------------------+---+



In [15]:
spark.conf.set("spark.sql.parquet.writeLegacyFormat", "true")
spark.conf.set("spark.sql.hive.convertMetastoreParquet", "false")
spark.conf.set("spark.sql.hive.convertMetastoreOrc", "false")
spark.conf.set("spark.datasource.writeDataSourceOptions", "true")


StatementMeta(, 97f41ba3-b063-4a96-abc5-bd27dccf330f, 17, Finished, Available, Finished)

In [16]:
df.write.format("delta").mode("overwrite").saveAsTable("inaturalist_speciescounts")

StatementMeta(, 97f41ba3-b063-4a96-abc5-bd27dccf330f, 18, Finished, Available, Finished)